# Notebook – Persoon 2 (Ismael)
## Project: Voorspelling Jeugdpopulatie Almere (CBS Wijken & Buurten)

**Doel van dit notebook:** Mijn taken uitvoeren volgens de teamplanning (CRISP-DM), met focus op filtering (Almere + wijkniveau), data understanding (15–25), feature engineering (ratio’s) en model-evaluatie (metrics).

> Let op: Dit template bevat bewust **geen code**. Ik voeg zelf code toe in de lege secties.

---
## Voorbereiding

- Bestand inlezen vanuit `data/raw/`.
- Zorg dat paden kloppen vanuit `notebooks/individual/`.
- Werk stapsgewijs en sla na mijn deliverables de juiste CSV op in `data/processed/`.

In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("../../data/raw/CBS_Kerncijfers.csv", sep=";", encoding="latin-1")
df.head()

,ï»¿ID,Peildatum,WijkenEnBuurten,Gemeentenaam_1,SoortRegio_2,Codering_3,IndelingswijzigingGemeenteWijkBuurt_4,AantalInwoners_5,Mannen_6,Vrouwen_7,...,PersonenPerSoortUitkeringAO_81,PersonenPerSoortUitkeringWW_82,PersonenPerSoortUitkeringAOW_83,JongerenMetJeugdzorgInNatura_84,PercentageJongerenMetJeugdzorg_85,WmoClienten_86,WmoClientenRelatief_87,TotaalDiefstalUitWoningSchuurED_88,VernielingMisdrijfTegenOpenbareOrde_89,GeweldsEnSeksueleMisdrijven_90
0,384,1-1-2025,GM0034,Almere,Gemeente,GM0034,1.0,229574,113738,115836,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,385,1-1-2025,WK003401,Almere,Wijk,WK003401,1.0,1810,880,930,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,386,1-1-2025,BU00340101,Almere,Buurt,BU00340101,1.0,795,415,385,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,387,1-1-2025,BU00340102,Almere,Buurt,BU00340102,1.0,1010,465,545,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,388,1-1-2025,WK003402,Almere,Wijk,WK003402,1.0,2890,1395,1500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


---
# Fase 1 – Data Structurering en Selectie

## Stap 1 – Filter Almere
**Doel:** Beperken tot alleen de gemeente Almere.

**Wat moet gebeuren:**
- Filter op `Gemeentenaam_1 = "Almere"`.
- Controleer of er geen andere gemeenten meer voorkomen.
- Vergelijk aantal rijen vóór en na filtering.

**Checks om te rapporteren:**
- Aantal rijen vóór filtering
- Aantal rijen na filtering
- Unieke waarden van `Gemeentenaam_1`

**Korte tekst (voor verslag):**
- Beschrijf kort wat er is gefilterd en waarom.

**Ruimte voor mijn code:**

**CSV opslaan als:**  
`data/processed/df_v1_raw_clean.csv`

**Toelichting:**  
Deze dataset bevat de originele CBS-data waarbij alleen technische fouten zijn opgelost (correcte encoding, juiste kolomscheiding, geen lege of foutieve kolommen).  
Er is inhoudelijk nog niets gefilterd.

## Stap 2 – Alleen wijkniveau selecteren (WK)
**Doel:** Alleen werken met wijken (geen buurten en geen gemeente-totaal).

**Wat moet gebeuren:**
- Filter op `SoortRegio_2 = "Wijk"`.
- Controleer of `Codering_3` begint met `WK`.
- Bepaal het aantal unieke wijken.

**Checks om te rapporteren:**
- Unieke waarden van `SoortRegio_2`
- Aantal unieke `Codering_3`
- Lijst (of tabel) met wijkcodes en wijknamen

**Ruimte voor mijn code:**


**CSV opslaan als:**  
`data/processed/df_v2_almere_wijken.csv`

**Toelichting:**  
Deze dataset bevat uitsluitend wijken (WK-codes) binnen de gemeente Almere.  
Gemeente- en buurtniveau zijn verwijderd.  
Alle originele variabelen zijn nog aanwezig.

## Stap 3 – Controle van tijdstructuur
**Doel:** Controleren of elke wijk meerdere jaren heeft en of de dataset bruikbaar is als tijdreeks.

**Wat moet gebeuren:**
- Controleer per wijk het aantal aanwezige jaren (via `Peildatum` of een `Jaar`-kolom als die al bestaat).
- Controleer of combinatie `Wijk (Codering_3) + Jaar` uniek is.
- Noteer opvallende gaten (missende jaren) of dubbele records.

**Checks om te rapporteren:**
- Aantal jaren per wijk (min/ max / gemiddeld)
- Aantal dubbele wijk-jaar combinaties

**Oplevering (CSV opslaan):**
- `data/processed/df_v2_almere_wijken.csv`

**Wat bevat deze CSV:**
- Alleen Almere
- Alleen wijkniveau (WK)
- Alle originele kolommen (nog geen kolomselectie)

**Ruimte voor mijn code:**


**CSV opnieuw opslaan als:**  
`data/processed/df_v2_almere_wijken.csv`

**Toelichting:**  
Na controle op dubbele wijk-jaar combinaties en aanwezigheid van meerdere jaren per wijk is de dataset gevalideerd als bruikbare tijdreeks.

---
# Fase 2 – Data Understanding

## Stap 4 – Groei 15–25 analyseren
**Doel:** Inzicht krijgen in de ontwikkeling van jongeren (15–25) per wijk door de tijd.

**Wat moet gebeuren:**
- Bekijk de trend van `k_15Tot25Jaar_9` per wijk over meerdere jaren.
- Bereken jaar-op-jaar verandering (verschil of percentage).
- Identificeer opvallende wijken of jaren (uitschieters).
- Lijngrafiek maken en vergelijken met 0-15 jaar.

**Ruimte voor mijn code:**


**CSV opslaan als:**  
`data/processed/df_v3_growth_analysis.csv`

**Toelichting:**  
Deze dataset bevat extra kolommen met jaar-op-jaar groei voor de leeftijdsgroep 15–25 jaar.  
De originele data blijft intact, maar er zijn analysekoppen toegevoegd.

## Stap 5 – Korte interpretatie (tekst)
**Doel:** Kort en duidelijk uitleggen wat de groei/krimp betekent.

**Vragen om te beantwoorden:**
- Groeien alle wijken ongeveer gelijk?
- Zijn er wijken met sterke stijging/daling?
- Zien we een breekpunt rond een bepaald jaar?
- Zijn de verschillen logisch/verklaarbaar?

**Mijn tekst (voor verslag):**
- (schrijf hier 5–10 zinnen)

---
# Fase 3 – Data Preparation

## Stap 6 – Ratio jongeren / totaal bevolking
**Doel:** Relatieve indicatoren maken zodat wijken eerlijker vergeleken kunnen worden.

**Wat moet gebeuren:**
- Maak ratio `0–15 / AantalInwoners` (op basis van `k_0Tot15Jaar_8` en `AantalInwoners_5`).
- Maak ratio `15–25 / AantalInwoners` (op basis van `k_15Tot25Jaar_9` en `AantalInwoners_5`).

**Waarom belangrijk:**
- Absolute aantallen groeien vaak mee met totale bevolkingsgroei.
- Ratio’s laten de echte demografische verschuiving zien.

**Checks om te rapporteren:**
- Min/max van ratio’s
- Zijn ratio’s logisch (tussen 0 en 1)?

**Ruimte voor mijn code:**


**CSV opslaan als:**  
`data/processed/df_v4_ratio_added.csv`

**Toelichting:**  
Deze dataset bevat nieuwe ratio-variabelen:
- Jongeren 0–15 / totaal inwoners  
- Jongeren 15–25 / totaal inwoners  

Hiermee kunnen wijken eerlijker worden vergeleken, onafhankelijk van bevolkingsgrootte.



## Stap 7 – Stabiliteit en plausibiliteit check
**Doel:** Controleren of de nieuwe features bruikbaar zijn.

**Wat moet gebeuren:**
- Controleer op onrealistische waarden (bijv. ratio > 1 of negatieve waarden).
- Check op null/NaN die ontstaan door deling (bijv. inwoners = 0).
- Noteer wat je aanpast/verwijdert en waarom.

**Oplevering (in teamverband):**
- De gezamenlijke output van fase 3 wordt opgeslagen als:
  - `data/processed/df_v6_model_ready.csv`

**Mijn bijdrage:**
- Ratio-features toegevoegd en gecontroleerd.

**Ruimte voor mijn code:**


### Oplevering – Definitieve Model Dataset

**CSV opslaan als:**  
`data/processed/df_v6_model_ready.csv`

**Toelichting:**  
Dit is de definitieve model-klaar gemaakte dataset.  
Deze bevat:
- Alleen relevante variabelen  
- Nieuwe features (ratio’s, groei, lag)  
- Geen dubbele rijen  
- Correcte datatypes  
- Alleen wijkniveau Almere  

Deze dataset wordt gebruikt voor train/test split en modeling.

---
# Fase 4 – Modeling (Evaluatie: mijn rol)

## Stap 8 – Modelmetrics berekenen
**Doel:** Beoordelen hoe goed het model voorspelt.

**Wat heb ik nodig:**
- `y_test` (echte waarden)
- `y_pred` (voorspelde waarden)

**Metrics:**
- MAE (Mean Absolute Error)
- RMSE (Root Mean Squared Error)
- R²

**Wat moet ik rapporteren:**
- De drie getallen (MAE/RMSE/R²)
- Korte interpretatie: is dit goed of slecht, en waarom?

**Ruimte voor mijn code:**


---
# Fase 5 – Evaluatie & Reflectie (mijn input)

**Kritische punten om te benoemen:**
- Is de tijdreeks lang genoeg (aantal jaren)?
- Heeft een wijkwijziging (indeling) invloed op trends?
- CBS-data bevat soms afrondingen/lege waarden; wat is de impact?
- Welke aannames maken we en wat betekent dat voor betrouwbaarheid?

**Mijn tekst (voor verslag):**
- (schrijf hier 8–12 zinnen)

---
## Checklist – Wat ik oplever

- `data/processed/df_v2_almere_wijken.csv` (na filtering Almere + wijkniveau)
- Input/observaties voor Data Understanding (15–25 trend)
- Ratio-features (0–15/inwoners en 15–25/inwoners) voor `df_v6_model_ready`
- Modelmetrics (MAE, RMSE, R²) zodra `y_test` en `y_pred` beschikbaar zijn

## Waarom elke stap een aparte CSV krijgt

Door elke fase als aparte dataset op te slaan:

- Is het proces reproduceerbaar  
- Kunnen fouten eenvoudig worden teruggedraaid  
- Is het CRISP-DM proces zichtbaar toegepast  
- Kan elke stap afzonderlijk worden gecontroleerd  
- Wordt direct modelleren op ruwe data voorkomen 